In [16]:
import os
import cv2
import cvlib as cv
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import load_model
import numpy as np
from matplotlib import pyplot as plt
from mtcnn import MTCNN
from keras.utils import np_utils
from sklearn.metrics import f1_score as sklearn_f1_score

In [10]:
def crop_images(input_dir, out_dir):
    source_list = os.listdir(input_dir)
    for f in source_list:
        f_name = os.path.join(input_dir, f)
        image = cv2.imread(f_name)

        # apply face detection
        faces, confidences = cv.detect_face(image)

        print(faces)
        print(confidences)

        # loop through detected faces
        for face, conf in zip(faces, confidences):
            (startX, startY) = face[0], face[1]
            (endX, endY) = face[2], face[3]
            # draw rectangle over face
            rectangle = cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

            # Check if the bounding box is within the image's dimensions
            if 0 <= startX < image.shape[1] and 0 <= endX < image.shape[1] and 0 <= startY < image.shape[0] and 0 <= endY < image.shape[0]:
                cropped_face = image[startY:endY, startX:endX]
                # save output
                cv2.imwrite(os.path.join(out_dir, f), cropped_face)
            else:
                print(f"The bounding box of face in {f} is out of the image's boundaries.")

crop_images('/data/FaceData/woman', "/data/FaceData/woman_croped/")

crop_images('/data/FaceData/man', "/data/FaceData/man_croped/")


[[197, 116, 413, 426]]
[0.9999559]
[[59, 41, 177, 197], [136, 123, 251, 272]]
[0.9999906, 0.9960616]
The bounding box of face in download (1).jpg is out of the image's boundaries.
[[69, 72, 147, 166]]
[0.99962497]
[[74, 5, 132, 83]]
[0.99676067]
[[93, 46, 171, 143]]
[0.9999738]
[[87, 32, 163, 140]]
[0.99924755]
[[52, 42, 168, 199], [128, 120, 242, 274]]
[0.9999573, 0.9978136]
The bounding box of face in download (6).jpg is out of the image's boundaries.
[[41, 39, 139, 184], [99, 130, 200, 273]]
[0.999928, 0.8085954]
The bounding box of face in download.jpg is out of the image's boundaries.
[[73, 28, 142, 130]]
[0.99999106]
[[192, 27, 268, 127], [27, 31, 106, 134]]
[0.99991834, 0.99930406]
[[59, 55, 126, 148]]
[0.9999901]
[[73, 36, 153, 150]]
[0.99967897]
[[34, 21, 151, 171], [113, 91, 229, 241]]
[0.9875519, 0.92625755]
The bounding box of face in images (13).jpg is out of the image's boundaries.
[[75, 30, 174, 177], [159, 90, 259, 242]]
[0.99963117, 0.93518806]
The bounding box of face

In [14]:
"""
Resizing Cropped Images and save them
"""
def resize_image(input_dir,output_dir):

    # specify the new dimensions you want for the images
    new_dimensions = (40, 40)  # you should change this to the dimensions you want

    # create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # add/modify image file types here
            img = Image.open(os.path.join(input_dir, filename))
            img.thumbnail(new_dimensions)
            img.save(os.path.join(output_dir, filename))

resize_image("/data/FaceData/man_croped/","/data/FaceData/man_resized/")

resize_image("/data/FaceData/woman_croped/","/data/FaceData/woman_resized/")

In [23]:
"""
Find F1 Score
"""

def compute_f1_score(test_generator, model):

    # Generate a list of labels
    test_labels = test_generator.classes
    # Convert the labels to categorical
    test_labels = np_utils.to_categorical(test_labels, num_classes=len(test_generator.class_indices))

    # Make the model predictions
    predictions = model.predict(test_generator)

    # Convert predictions classes to one hot vectors
    predicted_classes = np.argmax(predictions, axis=1)
    # Convert test observations to one hot vectors
    true_classes = np.argmax(test_labels, axis=1)

    # compute the confusion matrix
    f1Score = sklearn_f1_score(true_classes, predicted_classes, average='macro')

    print("F1 Score: ", f1Score)



In [28]:
"""
Train Model
"""
def train_model(train_dir,validation_dir,test_dir):

    # Define ImageDataGenerators for training, validation and testing
    # This will handle preprocessing, image augmentation, and splitting.
    datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

    validation_generator = datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

    test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

    # Create the model architecture.
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(len(train_generator.class_indices), activation='softmax'))

    # Compile the model.
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model.
    model.fit(train_generator, validation_data=validation_generator, epochs=10, batch_size=32)

    # Save the model for later use.
    model.save('model.h5')

    compute_f1_score(test_generator,model)

    return train_generator, validation_generator, test_generator, model



train_generator, validation_generator, test_generator, model = train_model("/data/FaceData/train","/data/FaceData/val","/data/FaceData/test")

Found 57 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Epoch 1/10
2/2 [==============================] - 2s 540ms/step - loss: 4.0557 - accuracy: 0.3860 - val_loss: 3.3883 - val_accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 1s 452ms/step - loss: 2.1397 - accuracy: 0.5088 - val_loss: 0.6487 - val_accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 1s 385ms/step - loss: 0.9792 - accuracy: 0.4737 - val_loss: 1.1582 - val_accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 1s 421ms/step - loss: 0.7571 - accuracy: 0.4912 - val_loss: 0.6385 - val_accuracy: 0.8333
Epoch 5/10
2/2 [==============================] - 1s 357ms/step - loss: 0.6175 - accuracy: 0.7544 - val_loss: 0.6771 - val_accuracy: 0.5000
Epoch 6/10
2/2 [==============================] - 1s 358ms/step - loss: 0.5242 - accuracy: 0.7719 - val_loss: 0.6812 - val_accuracy: 0.5000
Epoch 7/10
2/2 [==========================

In [29]:
"""
Load Model and predict face
"""
# Load the trained model
model = load_model('model.h5')

# Initialize the MTCNN detector for face detection
detector = MTCNN()


def predict_face(image_path, train_generator):
    # Load the image
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    # Use MTCNN to detect faces in the image
    result = detector.detect_faces(image)
    # If a face is detected
    if result:
        # Get the bounding box coordinates
        x, y, width, height = result[0]['box']
        # Crop the face from the image
        face = image[y:y + height, x:x + width]
        # Resize the face to the size your model expects
        face = cv2.resize(face, (150, 150))
        # Normalize the face
        face = face / 255.0
        # Expand dimensions to match the shape the model expects
        face = np.expand_dims(face, axis=0)

        # Use the model to predict the face
        prediction = model.predict(face)
        label = np.argmax(prediction)

        # Get the label name from the train generator's class_indices
        label_name = list(train_generator.class_indices.keys())[
            list(train_generator.class_indices.values()).index(label)]

        # Draw a rectangle around the face and add a label
        cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(image, label_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Convert the image color back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Save the image
        cv2.imwrite('/data/multi_class_face/val/jolie/1_edited.jpeg',
                    image)

        print(label_name)

        # Show the image with the face detected
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()




In [ ]:
# predict_face("/data/face/storm.jpg", train_generator)


In [18]:
"""
Face detection with video
"""

def predict_face_video(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    while(video.isOpened()):
        # Read a frame
        ret, frame = video.read()
        if not ret:
            break

        # Convert the frame color
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use MTCNN to detect faces in the frame
        result = detector.detect_faces(frame_rgb)

        for i in range(len(result)):
            # Get the bounding box coordinates
            x, y, width, height = result[i]['box']
            # Crop the face from the frame
            face = frame_rgb[y:y + height, x:x + width]
            # Resize the face to the size your model expects
            face = cv2.resize(face, (150, 150))
            # Normalize the face
            face = face / 255.0
            # Expand dimensions to match the shape the model expects
            face = np.expand_dims(face, axis=0)

            # Use the model to predict the face
            prediction = model.predict(face)
            label = np.argmax(prediction)

            # Get the label name from the train generator's class_indices
            label_name = list(train_generator.class_indices.keys())[
                list(train_generator.class_indices.values()).index(label)]

            # Draw a rectangle around the face and add a label
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv2.putText(frame, label_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Video', frame)

        # Break the loop on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the video capture and video write objects
    video.release()

    # Close all the frames
    cv2.destroyAllWindows()


predict_face_video('/data/face_video/20230519_131703.mp4')


1/1 [==============================] - 0s 17ms/step


C:\Users\mupaj\Documents\Visual Computing\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\mupaj\Documents\Visual Computing\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\mupaj\Documents\Visual Computing\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\mupaj\Documents\Visual Computing\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 

In [32]:
"""
Detect Face via webcam
"""

def predict_face_webcam():
    # Open the webcam
    video = cv2.VideoCapture(0)

    while True:
        # Read a frame
        ret, frame = video.read()

        # Convert the frame color
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use MTCNN to detect faces in the frame
        result = detector.detect_faces(frame_rgb)

        for i in range(len(result)):
            # Get the bounding box coordinates
            x, y, width, height = result[i]['box']
            # Crop the face from the frame
            face = frame_rgb[y:y + height, x:x + width]
            # Resize the face to the size your model expects
            face = cv2.resize(face, (150, 150))
            # Normalize the face
            face = face / 255.0
            # Expand dimensions to match the shape the model expects
            face = np.expand_dims(face, axis=0)

            # Use the model to predict the face
            prediction = model.predict(face)
            label = np.argmax(prediction)

            # Get the label name from the train generator's class_indices
            label_name = list(train_generator.class_indices.keys())[
                list(train_generator.class_indices.values()).index(label)]

            # Draw a rectangle around the face and add a label
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv2.putText(frame, label_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Video', frame)

        # Break the loop on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the video capture
    video.release()

    # Close all the frames
    cv2.destroyAllWindows()


In [33]:
predict_face_webcam()


1/1 [==============================] - 0s 19ms/step


C:\Users\mupaj\Documents\Visual Computing\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\mupaj\Documents\Visual Computing\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\mupaj\Documents\Visual Computing\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\mupaj\Documents\Visual Computing\venv\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 